<a href="https://colab.research.google.com/github/vasconce1o/MonteCarloDropout/blob/main/monte_carlo_dropout_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XI TALLER INTERNACIONAL DE CIBERNÉTICA APLICADA
# **Comprendiendo Monte Carlo Dropout para la representación de la incertidumbre en modelos de Aprendizaje Profundo**

**Autor**: Yuniesky Orlando Vasconcelo Mir, yovasconcelo@xetid.cu




Después de entrenar durante aproximadamente 10 épocas, este modelo obtiene una precisión del 90,7% en el conjunto de pruebas. Para activar el dropout en el momento de la predicción, simplemente necesitamos establecer training=True comportamiento similar al entrenamiento, es decir, eliminar algunas neuronas. De esta manera, cada predicción será ligeramente diferente y podremos generar tantas como queramos.

In [ ]:
from tensorflow import  keras

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=30)
acc1 = model.evaluate(X_test, y_test)

Epoch 1/30
1875/1875 [==============================] - 10s 5ms/step - loss: 4.0323 - accuracy: 0.7576
Epoch 2/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5417 - accuracy: 0.8543
Epoch 3/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4103 - accuracy: 0.8889
Epoch 4/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3652 - accuracy: 0.9035
Epoch 5/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3416 - accuracy: 0.9101
Epoch 6/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3169 - accuracy: 0.9156
Epoch 7/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3137 - accuracy: 0.9192
Epoch 8/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2972 - accuracy: 0.9231
Epoch 9/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2956 - accuracy: 0.9246
Epoch 10/30
1875/1875 [==============================] - 9s 5ms/step - loss: 0.29

Creando dos funciones útiles: predict_proba() genera el número num_samples deseado de predicciones y promedia la probabilidad de clase predicha para cada uno de los 10 dígitos en el conjunto de datos MNIST, mientras que predict_class() simplemente elige la probabilidad predicha más alta para elegir la clase más probable.

In [ ]:
def predict_proba(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.stack(preds).mean(axis=0)
     
def predict_class(X, model, num_samples):
    proba_preds = predict_proba(X, model, num_samples)
    return np.argmax(proba_preds, axis=1)

Ahora, hagamos 100 predicciones y evaluemos la precisión en el conjunto de pruebas.

In [ ]:
import numpy as np
y_pred = predict_class(X_test, model, 100)
acc = np.mean(y_pred == y_test)
print(acc)

0.9727


Esto arroja una precisión del 97,2%. En comparación con el resultado anterior, hemos disminuido la tasa de error del 3,3% al 2,8%, que es un 15%. Sin cambiar ni volver a entrenar el modelo.

# Monte Carlo Dropout
 Un vistazo a la incertidumbre de la predicción. 
 En las tareas de clasificación, las probabilidades de clase obtenidas de la salida softmax a menudo se interpretan erróneamente como confianza del modelo. Sin embargo, Gal y Ghahramani (2016) muestran que un modelo puede ser incierto en sus predicciones incluso con una alta producción de softmax. También se puede observar en las predicciones de MNIST. Comparese la salida softmax con las probabilidades predichas por Monte Carlo Dropout para un solo ejemplo de prueba.

In [ ]:
y_pred_proba = predict_proba(X_test, model, 100)

softmax_output = np.round(model.predict(X_test[1:2]), 3)
mc_pred_proba = np.round(y_pred_proba[1], 3)
print(softmax_output, mc_pred_proba)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]] [0.029 0.002 0.956 0.003 0.002 0.001 0.    0.002 0.004 0.001]


Ambos están de acuerdo en que el ejemplo de prueba es muy probable que sea de la 3ª clase. Sin embargo, el softmax está 100% seguro de que ese es el caso, lo que ya debería alertarte de que algo no está bien. Las estimaciones de probabilidad de 0% o 100% suelen ser peligrosas. Monte Carlo Dropout nos proporciona mucha más información sobre la incertidumbre de la predicción: lo más probable es que sea de clase 3, pero hay una pequeña posibilidad de que sea de clase 4, y 5, aunque poco probable, sigue siendo más probable que 1, por ejemplo.